In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

2.5.0


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model
from tensorflow.compat.v1.keras.layers import Dense, Bidirectional, Input,Dropout,BatchNormalization, CuDNNGRU, CuDNNLSTM

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",100)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from gensim.models import word2vec
import tensorflow as tf
from tensorflow import keras

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from keras.layers import Dense, Dropout, LSTM, Embedding, Conv1D, MaxPooling1D, Bidirectional
from keras.models import Sequential

from sklearn.metrics import classification_report
from keras.models import Sequential

import warnings
warnings.filterwarnings("ignore")

from keras.models import Model
from keras.layers import Input, Embedding, Dense,concatenate
from keras.layers import Reshape, BatchNormalization, Flatten, Concatenate, Dropout, SpatialDropout1D

from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm

In [ ]:
from gensim.models import fasttext
data = pd.read_pickle("gdrive/MyDrive/Colab Notebooks/data/clean_data_komoran_0501_noun_adj.pkl")
w2v_model = word2vec.Word2Vec.load("gdrive/MyDrive/Colab Notebooks/data/w2v_model_komoran_0501_noun")  # 불러오기
final_data = pd.read_pickle("gdrive/MyDrive/Colab Notebooks/data/final_data_total.pkl")

In [ ]:
model_data = pd.merge(final_data , data[['code','comment_token']], how = 'left', on = "code")
model_data['target'].value_counts()

0    17604
1      140
Name: target, dtype: int64

In [ ]:
data_0 = model_data[model_data['target']==0].sample(200, random_state = 42)
data_1 = model_data[model_data['target']==1]
model_data = pd.concat([data_0,data_1], axis = 0, ignore_index= True)

In [ ]:
'''
total_token_lst = []
for token_value in model_data['comment_token']:
  token_lst = []
  for value_ in token_value:
    if len(value_) > 2:
      token_lst.append(value_) 
  total_token_lst.append(token_lst)
  '''

"\ntotal_token_lst = []\nfor token_value in model_data['comment_token']:\n  token_lst = []\n  for value_ in token_value:\n    if len(value_) > 2:\n      token_lst.append(value_) \n  total_token_lst.append(token_lst)\n  "

In [ ]:

# 모든 글자를 정수 ID로 인코딩
def word_to_id(token_list):
    print("sample data : ",token_list[0])
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=500000)
    tokenizer.fit_on_texts(token_list)
    print("texts to sequences ===>",tokenizer.texts_to_sequences([token_list[0]]))
    print("sequences to texts ===>",tokenizer.sequences_to_texts([[1,2,3,4,5]])) 
    print()
    
    word_to_id = tokenizer.word_index
    id_to_word = tokenizer.index_word
    
    print("고유 단어 개수 : {}, 전체 청원 글 개수 : {}".format(len(word_to_id), tokenizer.document_count))
    
    encoded_tokens = []
    for token in token_list:
        encoded_tokens.append(tokenizer.texts_to_sequences([token])[0])
    
    return encoded_tokens, word_to_id, id_to_word

# with pretrained model
def create_embedding_matrix(w2v_model, word_to_id):
    '''
    pretrained model이 존재할 때 embedding matrix 만드는법
    if 존재하지 않는다면, tf.keras.Embedding layer 사용
    '''
    EMBEDDING_DIM = w2v_model.vector_size

    embedding_matrix = np.zeros((len(word_to_id)+1, EMBEDDING_DIM))
    print(embedding_matrix.shape)

    for word, i in word_to_id.items():
        try:
            embedding_vector = w2v_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i] = np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
            
    return embedding_matrix


In [ ]:
# train_token_list = total_token_lst
train_token_list = model_data['comment_token']
        
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=500000)
tokenizer.fit_on_texts(train_token_list)

word_to_id = tokenizer.word_index
id_to_word = tokenizer.index_word

encoded_tokens = []

for token in train_token_list:
    encoded_tokens.append(tokenizer.texts_to_sequences([token])[0])

print("고유 단어 개수 : {}, 전체 청원 글 개수 : {}".format(len(word_to_id), tokenizer.document_count))

# X (padding)
MAX_SEQUENCE_LENGTH = 100

EMBEDDING_DIM = w2v_model.vector_size

embedding_matrix = np.zeros((len(word_to_id)+1, EMBEDDING_DIM))

for word, i in word_to_id.items():
    try:
        embedding_vector = w2v_model.wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i] = np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)


data = tf.keras.preprocessing.sequence.pad_sequences(encoded_tokens, maxlen=MAX_SEQUENCE_LENGTH)
labels_ = model_data['target']


고유 단어 개수 : 9406, 전체 청원 글 개수 : 340


In [ ]:
data

array([[   0,    0,    0, ..., 1258,  176,  852],
       [ 539,  187,  540, ...,  492,  345, 1560],
       [   0,    0,    0, ...,   60, 2569,   84],
       ...,
       [   0,    0,    0, ...,   16, 1273,  143],
       [ 505,  361,  959, ...,   16,  626,  143],
       [1476, 2291, 3771, ...,  858,  895, 5745]], dtype=int32)

In [ ]:
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)

# data = data[indices]
# labels = labels[indices]

model_data["len_title"] = model_data['title'].apply(lambda x: len(x))
model_data["len_content"] = model_data['content'].apply(lambda x: len(x))
model_data["noun_count"] = model_data['doc_noun_list'].apply(lambda x: len(x))

X_train, X_test, y_train_, y_test_ = train_test_split(data, labels_, stratify = labels_, shuffle = True, random_state = 42, test_size = 0.3)
cat_encoder = OneHotEncoder()
y_train = cat_encoder.fit_transform(pd.DataFrame(y_train_, columns = ['target'])[['target']]).toarray()
y_test = cat_encoder.fit_transform(pd.DataFrame(y_test_, columns = ['target'])[['target']]).toarray()

In [ ]:
y_train_

127    0
137    0
109    0
244    1
283    1
      ..
93     0
263    1
145    0
42     0
98     0
Name: target, Length: 238, dtype: int64

In [ ]:
from sklearn.utils import class_weight

vocab_size, embedding_dim = embedding_matrix.shape

In [ ]:
import random

class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(config.seed)

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=30)]

In [ ]:
EMBEDDING_DIM = 300

model = Sequential()

model.add(Embedding(vocab_size, EMBEDDING_DIM,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False))
model.add(CuDNNLSTM(128, return_sequences= True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(64))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=30)]

hist = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split = 0.1, verbose = 1, callbacks=callbacks, class_weight = class_weights)

print(X_train.shape)
print(X_test.shape)

pred_train = []
value_lst = model.predict(X_train)
for i in range(len(value_lst)):
    pred_train.append(np.argmax(value_lst[i]))

y_train_result = []
for i in range(len(y_train)):
    y_train_result.append(np.argmax(y_train[i]))

pred_test = []
value_lst = model.predict(X_test)
for i in range(len(value_lst)):
    pred_test.append(np.argmax(value_lst[i]))

y_test_result = []
for i in range(len(y_test)):
    y_test_result.append(np.argmax(y_test[i]))

print(classification_report(pred_train, y_train_result))
print(classification_report(pred_test, y_test_result))
print(confusion_matrix(pred_test, y_test_result))

Epoch 1/100
4/4 [==============================] - 2s 187ms/step - loss: 0.6866 - accuracy: 0.5246 - val_loss: 0.6881 - val_accuracy: 0.5833
Epoch 2/100
4/4 [==============================] - 0s 29ms/step - loss: 0.6385 - accuracy: 0.6996 - val_loss: 0.6978 - val_accuracy: 0.5000
Epoch 3/100
4/4 [==============================] - 0s 26ms/step - loss: 0.5704 - accuracy: 0.7591 - val_loss: 0.7395 - val_accuracy: 0.5417
Epoch 4/100
4/4 [==============================] - 0s 24ms/step - loss: 0.4986 - accuracy: 0.7589 - val_loss: 0.8523 - val_accuracy: 0.5417
Epoch 5/100
4/4 [==============================] - 0s 24ms/step - loss: 0.4366 - accuracy: 0.7946 - val_loss: 0.8353 - val_accuracy: 0.5833
Epoch 6/100
4/4 [==============================] - 0s 25ms/step - loss: 0.3595 - accuracy: 0.8443 - val_loss: 0.8699 - val_accuracy: 0.5833
Epoch 7/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3361 - accuracy: 0.8566 - val_loss: 1.0126 - val_accuracy: 0.5417
Epoch 8/100
4/4 [==

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True)

acc_per_fold = []
loss_per_fold = []

idx = 0
vocab_size, embedding_dim = embedding_matrix.shape

for train_index, test_index in kf.split(data):
  
  print(train_index)
  print(test_index)
  X_train, X_test = data[train_index], data[test_index]
  y_train, y_test = labels[train_index], labels[test_index]
  y_train_class = model_data['target'][train_index]

  class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y_train_class),
                                                  y_train_class)

  class_weights = {0:class_weights[0], 1:class_weights[1]}


  print(X_train.shape)
  print(y_train.shape)
  
  print(X_test.shape)
  print(y_test.shape)
  
  EMBEDDING_DIM = 300

  model = Sequential()

  model.add(Embedding(vocab_size, EMBEDDING_DIM,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False))
  model.add(CuDNNLSTM(128, return_sequences= True))
  model.add(Dropout(0.2))
  model.add(CuDNNLSTM(64))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=30)]

  hist = model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split = 0.1, verbose = 1, callbacks=callbacks, class_weight = class_weights)


  pred_train = []
  value_lst = model.predict(X_train)
  for i in range(len(value_lst)):
      pred_train.append(np.argmax(value_lst[i]))

  y_train_ = []
  for i in range(len(y_train)):
      y_train_.append(np.argmax(y_train[i]))

  pred_test = []
  value_lst = model.predict(X_test)
  for i in range(len(value_lst)):
      pred_test.append(np.argmax(value_lst[i]))

  y_test_ = []
  for i in range(len(y_test)):
      y_test_.append(np.argmax(y_test[i]))

  print(classification_report(pred_train, y_train_))
  print(classification_report(pred_test, y_test_))
  print(confusion_matrix(pred_test, y_test_))
  
  scores = model.evaluate(X_test, y_test, verbose = 0)
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])


[  1   2   3   4   5   6   8   9  10  11  12  13  14  15  16  19  21  22
  24  25  27  28  29  30  31  32  33  34  35  36  37  38  39  44  45  46
  47  49  50  51  53  55  56  57  58  59  60  61  62  63  64  66  67  69
  70  71  72  73  74  75  76  78  79  80  81  82  83  85  87  88  89  90
  91  92  93  95  96  97  98  99 100 102 103 104 106 107 108 109 110 111
 112 113 114 116 119 121 122 123 124 130 131 132 134 135 136 137 138 139
 140 141 142 144 145 146 147 149 150 151 152 153 154 155 157 158 159 160
 162 163 164 165 167 168 169 170 171 172 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 192 193 194 196 197 198 199 200 202 203 204 205
 206 207 208 209 211 213 214 215 217 218 219 220 222 223 225 226 228 230
 231 233 234 235 236 237 238 242 243 244 245 246 248 249 250 251 252 253
 254 255 257 258 259 260 261 262 263 265 267 268 269 270 271 272 273 274
 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 291 292 293
 296 297 298 299 300 302 303 304 307 308 309 310 31

In [ ]:
vocab_size, embedding_dim = embedding_matrix.shape

def cudnnlstm_model_():
    inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    meta_input = Input(shape = (1,), name = 'meta_input')

    x = Embedding(vocab_size, EMBEDDING_DIM,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable = False)(inp)
    
    x_f = CuDNNLSTM(128, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)    
    x_b = CuDNNLSTM(128, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)   
    
    x = concatenate([x_f, x_b])
    x = Dropout(0.2)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(2, activation="softmax")(x)
    model = Model(inputs=[inp, meta_input], outputs=x)

    return model

cudnnlstm_model = cudnnlstm_model_()
cudnnlstm_model.summary()


Model: "model_106"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_111 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_153 (Embedding)       (None, 100, 300)     2822100     input_111[0][0]                  
__________________________________________________________________________________________________
cu_dnnlstm_310 (CuDNNLSTM)      (None, 128)          220160      embedding_153[0][0]              
__________________________________________________________________________________________________
cu_dnnlstm_311 (CuDNNLSTM)      (None, 128)          220160      embedding_153[0][0]              
__________________________________________________________________________________________

In [ ]:
X_train_meta = model_data[['len_title', 'len_content','sim_count']].iloc[y_train_.index]
X_test_meta = model_data[['len_title','len_content','sim_count']].iloc[y_test_.index]


cat_encoder = OneHotEncoder()
y_train = cat_encoder.fit_transform(pd.DataFrame(y_train_, columns = ['target'])[['target']]).toarray()
y_test = cat_encoder.fit_transform(pd.DataFrame(y_test_, columns = ['target'])[['target']]).toarray()

# X_train_meta = model_data[['sim_count']].iloc[train_index]
# X_test_meta = model_data[['sim_count']].iloc[test_index]

cudnnlstm_model = cudnnlstm_model_()
cudnnlstm_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=30)]

hist = cudnnlstm_model.fit([X_train, X_train_meta], y_train, batch_size = 64, epochs = 100, validation_split = 0.1, verbose = 1, callbacks=callbacks, class_weight = class_weights)

pred_train = []
value_lst = cudnnlstm_model.predict([X_train, X_train_meta])
for i in range(len(value_lst)):
    pred_train.append(np.argmax(value_lst[i]))

y_train_result = []
for i in range(len(y_train)):
    y_train_result.append(np.argmax(y_train[i]))

pred_test = []
value_lst = cudnnlstm_model.predict([X_test, X_test_meta])
for i in range(len(value_lst)):
    pred_test.append(np.argmax(value_lst[i]))

y_test_result = []
for i in range(len(y_test)):
    y_test_result.append(np.argmax(y_test[i]))


print(classification_report(pred_train, y_train_result))
print(classification_report(pred_test, y_test_result))
print(confusion_matrix(pred_test, y_test_result))


Epoch 1/100
4/4 [==============================] - 2s 199ms/step - loss: 0.6988 - accuracy: 0.4787 - val_loss: 0.7021 - val_accuracy: 0.4583
Epoch 2/100
4/4 [==============================] - 0s 34ms/step - loss: 0.6540 - accuracy: 0.5632 - val_loss: 0.6956 - val_accuracy: 0.5417
Epoch 3/100
4/4 [==============================] - 0s 31ms/step - loss: 0.6216 - accuracy: 0.6481 - val_loss: 0.6876 - val_accuracy: 0.5417
Epoch 4/100
4/4 [==============================] - 0s 26ms/step - loss: 0.5519 - accuracy: 0.7625 - val_loss: 0.6902 - val_accuracy: 0.5417
Epoch 5/100
4/4 [==============================] - 0s 25ms/step - loss: 0.4810 - accuracy: 0.8014 - val_loss: 0.7902 - val_accuracy: 0.5833
Epoch 6/100
4/4 [==============================] - 0s 23ms/step - loss: 0.4215 - accuracy: 0.8422 - val_loss: 0.8468 - val_accuracy: 0.5417
Epoch 7/100
4/4 [==============================] - 0s 22ms/step - loss: 0.4102 - accuracy: 0.8184 - val_loss: 0.7726 - val_accuracy: 0.5417
Epoch 8/100
4/4 [==